# **Memory For Smart RAG System == Combined Memory**

In [1]:
import os
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()
# Get the value of the environment variable
openai_api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


In [3]:
from langchain.memory import (ConversationBufferMemory, 
                              ConversationSummaryMemory,
                              CombinedMemory)

In [4]:
llm = ChatOpenAI(temperature=0.7)

In [5]:
buffer_memory = ConversationBufferMemory(memory_key="buffer_history", return_messages=True)

summary_memory = ConversationSummaryMemory(llm=llm,
                                            memory_key="summary_history",
                                            return_messages=True)

C:\Users\vikas\AppData\Local\Temp\ipykernel_29356\3373453380.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  buffer_memory = ConversationBufferMemory(memory_key="buffer_history", return_messages=True)
C:\Users\vikas\AppData\Local\Temp\ipykernel_29356\3373453380.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  summary_memory = ConversationSummaryMemory(llm=llm,


In [6]:
combined_memory = CombinedMemory(
    memories=[buffer_memory, summary_memory],
    memory_key="combined_history",
    return_messages=True
)

c:\Users\vikas\OneDrive\Desktop\Complete-Langchain-By-Euron\langchain\lib\site-packages\langchain\memory\combined.py:40: UserWarning: When using CombinedMemory, input keys should be so the input is known.  Was not set on chat_memory=InMemoryChatMessageHistory(messages=[]) return_messages=True memory_key='buffer_history'
  warnings.warn(
c:\Users\vikas\OneDrive\Desktop\Complete-Langchain-By-Euron\langchain\lib\site-packages\langchain\memory\combined.py:40: UserWarning: When using CombinedMemory, input keys should be so the input is known.  Was not set on llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001E4620B88B0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001E4620B8550>, root_client=<openai.OpenAI object at 0x000001E4620BBB20>, root_async_client=<openai.AsyncOpenAI object at 0x000001E4620BA9E0>, temperature=0.7, model_kwargs={}, openai_api_key=SecretStr('**********')) chat_memory=InMemor

In [7]:
prompt = PromptTemplate(
    input_variables= ["buffer_history", "summary_history", "input"],
    template="""
You are a helpful assistant.

Recent conversation:
{buffer_history}

Summary:
{summary_history}

Human: {input}
AI:
"""
)

In [8]:
def run_conversation(user_input : str):
    memory_vars = combined_memory.load_memory_variables({"inputs": user_input})
    
    full_input = {
        "input": user_input,
        **memory_vars
    }
    
    prompt_str = prompt.format(**full_input)
    
    response = llm.invoke(prompt_str)
    
    combined_memory.save_context({"input": user_input}, {"output": response.content})
    return response.content

In [9]:
print(run_conversation("Hello, how are you, My name is Vikas Jangid."))
print(run_conversation("I am a AI/ML engineer?"))
print(run_conversation("I am 24 year old?"))


Hello Vikas Jangid! I'm here to assist you. How can I help you today?
That's great! How can I assist you with your work as an AI/ML engineer?
That's great to know! Is there anything specific you would like assistance with or any questions you have related to being a 24-year-old AI/ML engineer?


In [10]:
print(run_conversation("Hello, tell me about myself?"))

You are Vikas Jangid, a 24-year-old AI/ML engineer.


In [11]:
print("The End")

The End
